# INFO 2950: Phase IV
#### Group Members: Anusha Bishayee, Katheryn Ding

---
### __Introduction:__  

#### How do ESG score and stock performance (price) align across different industries? What associations can we find between company industry, stock performance, and ESG ratings?
#### note: ESG score refers to a quantiative metric measuring a company's environmental, social, and governance performance; 'environmental' pertains to aspects like waste management and energy emissions, 'social' pertains to aspects like customer satisfaction and DEI in the workplace, and 'governance' pertains to aspects like operating efficiencies and risk management. ESG scores are typically examined by independent investors, business analysts, and even competitior companies to assess risk or opportunities associated with a specific company's practices.



---
### __Data Collection and Cleaning:__

In [1]:
import contextlib
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import yfinance as yf
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy.stats import norm

the data cleaning work that produces these csvs is found here: https://github.com/anushabishayee/info2950_finalproject/blob/main/data_cleaning.ipynb.

Generally, we imported the yfinance library, downloaded the Kaggle csv with the company ESG data, cleaned the dataset for NaNs and unneeded values, and reformatted some date values for ease of manipulation. Then, we joined the ESG data to the yfinance stock data, matching on company ticker (we created 4 new stock metric columns), and dropped NaNs for the creation of merged_df. For the 2nd dataset (sample_companies), we stratified and randomly selected 30 specific companies from this main dataset (specific methodology is outlined above in the 'Data Collection and Cleaning' section. For the 3rd dataset, we extracted the data straight from the yfinance library, and calculated a rate of change variable for the stock change as well for sp500.

In [2]:
avg_esg_by_industry = pd.read_csv("avg_esg_by_industry.csv")
sp500 = pd.read_csv("sp500.csv")
merged_df = pd.read_csv("merged_df.csv")
sample_companies = pd.read_csv("sample_companies.csv")

---
### __Data Description__
1) Where can your raw source data be found, if applicable? Provide a link to the raw data (hosted on Github, in a Cornell Google Drive or Cornell Box).

- We have 3 main datasets: 1 main dataset (merged_df), 1 "sample" dataset that selects 30 rows from this main dataset (sample_companies), and 1 additional dataset (sp500). Our raw data for the first 2 datasets can be found on Kaggle, here: https://github.com/anushabishayee/info2950_finalproject/blob/main/raw%20data/esg%20kaggle%20csv%20link and the actual csv is here: https://github.com/anushabishayee/info2950_finalproject/blob/main/esg_data.csv.
- Even more specifically, the Kaggle author states that they pulled the data for their csv from multiple APIs, like ESG Enterprise, a publicly-available API. They grabbed financial and company data from Finnhub. 3 of these links can be found here: https://github.com/anushabishayee/info2950_finalproject/blob/main/raw%20data/esg%20kaggle%20source%20data%20links.
- The finance data that the 3rd dataset is comprised of, and the finance data that is joined to the 1st and 2nd datasets is found in the yfinance library in Python (Yahoo Finance data, https://github.com/anushabishayee/info2950_finalproject/blob/main/raw%20data/yfinance%20links).

***

2) If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?

- Individuals are not involved in the data directly, as each observation corresponds to an entire company.

***

3) What preprocessing was done, and how did the data come to be in the form that you are using?

- Our preprocessing of these datasets is detailed above. Generally, we imported the yfinance library, downloaded the Kaggle csv with the company ESG data, cleaned the dataset for NaNs and unneeded values, and reformatted some date values for ease of manipulation. Then, we joined the ESG data to the yfinance stock data, matching on company ticker (we created 4 new stock metric columns), and dropped NaNs for the creation of merged_df. For the 2nd dataset (sample_companies), we stratified and randomly selected 30 specific companies from this main dataset (specific methodology is outlined above in the 'Data Collection and Cleaning' section. For the 3rd dataset, we extracted the data straight from the yfinance library, and calculated a rate of change variable for the stock change as well for sp500.
- For the Kaggle csv, the author notes that they used company stock ticker as a unique identifier, then pulled and collated data from various APIs. in specific, they utilized ESG Enterprise (https://www.esgenterprise.com/), a publicly-available API, and pulled their ratings methodology from https://app.esgenterprise.com/uploads/ESG-Enterprise-Risk-Ratings-MethodologyV3.pdf. They grabbed financial and company data from Finnhub (https://finnhub.io/).

***

4) What processes might have influenced what data was observed and recorded and what was not?

- For the ESG data, the Kaggle author of the csv specifically mentioned that only mid/large-cap companies are included, so this influences the specific companies that are recorded in the initial data - smaller companies (that also might have an ESG score) will not be 'observed' here. The author pulled data from ESG Enterprise and Finnhub, so any companies that do not have data available there will not be observed in the dataset. We also dropped any company that had a NaN or blank column value for the ESG columns, and dropped any company that didn't have stock data available in Yahoo Finance (or had NaNs for any specific finance column).

***

5) Who funded the creation of the dataset?

- We created these 3 analysis-ready datasets from two data sources: a 'Public Company ESG Ratings Dataset' Kaggle dataset from user Alistair King (https://www.kaggle.com/alistairking), a New York-based Kaggle Datasets Grandmaster, as well as the yfinance Python library, created by Ran Aroussi (https://aroussi.com/) as a way around the 2017 Yahoo Finance API deprecation. It is unclear if these datasets were 'funded', but their organization and accumulation were spearheaded by the two aforementioned people, respectively.

***

6) Why was this dataset created?
- We formulated our main analysis-ready dataset (merged_df) to examine associations between some of the largest USD-utilizing companies' ESG scores and their stock performances (as well as industry-specific analyses). Then, we formulated our sample dataset (sample_companies) so that we could take a look at some company-level analyses of the general data and research question (620 companies are kinda hard to visualize simultaneously). Finally, we formulated the sp500 dataset so that we could contrast company stock performance from the specified range of 2/1/21 - 12/31/22 to the overall performance of the S&P 500. (the rationale for the range of 2/1/21 - 12/31/22 is mentioned above, it's due to the fact that most companies have a 'last processing date' of February 2022 - Novermber 2022 for their ESG score.)
- The original ESG csv was created and uploaded by Kaggle user Alistair King, perhaps for personal enrichment or curiosity (they do have a Kaggle Datasets Grandmaster rank, so perhaps they just enjoy creating and uploading datasets). The original yfinance Python library was created by Ran Aroussi to have a simple way to download historical market data from Yahoo Finance, due to the Yahoo Finance API deprecation.

***

7) What are the observations (rows) and attributes (columns)?
- For the S&P 500 dataset (sp500), the rows each correspond to a specific date where the S&P 500 was measured, within the range from 2/1/21 - 12/31/22. The columns for this dataset are Start Price, End Price, and Rate of Change (aka the starting price of the S&P 500 when the US market opened on a specific day, the ending price of the S&P 500 when the US market closed on the same specific day, and the percentage change that this stock exhibited between the start and close times of that specific day (100 * (end price - start price) / start price)).

- For both the merged_df and sample_companies dataset, each row corresponds to an unique, mid- to large-cap company that is publicly-traded and utilizes USD. merged_df, our main dataset, has 620 companies, while sample_companies has 30 companies for now. merged_df and sample_companies have the same columns, they are:
+ ticker - a unique combo of letters and numbers that represent a particular stock
+ name - the official name of the company
+ currency - the currency the company is traded in (this was filtered to only be USD)
+ exchange - what market the company is exchanged on
+ industry - the type of output the company produces
+ logo - a link to the company logo, potentially for joining with other datasets (MIGHT BE DROPPED LATER)
+ weburl - a link to the company website, potentially for joining with other datasets or scraping for text sentiment analysis (MIGHT BE DROPPED LATER)
+ environment_grade - a letter score given to the company that measures how well it complies to environmental standards, ranging from AAA being the best to CCC being the worst
+ environment_level - a categorical classification of a company's overall environmental performance (low, medium, high)
+ social_grade - a letter score given to the company that measures how well it complies to social standards, ranging from AAA being the best to CCC being the worst
+ social_level - a categorical classification of a company's overall social performance (low, medium, high)
+ governance_grade - a letter score given to the company that measures how well it complies to governance standards, ranging from AAA being the best to CCC being the worst
+ governance_level - a categorical classification of a company's overall governance performance (low, medium, high)
+ environment_score - a numerical measure of how well a company performs on environment-related factors, ranging from 0-1000
+ social_score - a numerical measure of how well a company performs on social-related factors, ranging from 0-1000
+ governance_score - a numerical measure of how well a company performs on governance-related factors, ranging from 0-1000
+ total_score - a numerical measure of how well a company performs on environment, social, and governance-related factors, ranging from 0-1500]
+ cik - central index key, a unique identifier assigned by the SEC to any company that files documents with the SEC (MIGHT BE DROPPED LATER)
the following columns are ones that we created, using the yfinance data:
+ percent_change - the percent change in the company stock price from close time on 2/1/21 to close time on 12/31/22 (100 * (end price - start price) / start price))
+ start_close - the closing price of the company stock on 2/1/21
+ end_close - the closing price of the company stock on 12/31/22
+ volatility - standard deviation of daily returns of the company stock, aka the percentage change in the stock price from day to day (indicator of how much stock price fluctuates in a given period, higher volatility is riskier, lower volatility has more stability). specifically, daily return is calculated by closing price on day x+1 - closing price on day x divided by closing price on day x, so all daily returns in the time period 2/1/21-12/31/22 are calculated for the specific company stock, and then the standard deviation is taken to get the volatility
+ 50_day_SMA - 50 day simple moving average, or the sum of closing price of a company stock for the last 50 days before 12/31/22, divided by 50 (if current stock price is above the 50-day SMA, the company is in uptrend, and vice versa)
+ cumulative_return - cumulative return of the company stock over the entire period ((close price on 12/31/22 / close price on 2/1/21) - 1), positive values represent returns, and negative values represent losses

---
### __Data Limitations__

1. ESG is typically evaluated annually, which might mean the scores in our dataset don't reflect the most accurate performance of the company, which directly impacts the analyses and conclusions we might draw from our EDA. in other words, when considering the short-term impact of the company's ESG and other policies, it's likely that policy change affect stocks immediately, but these changes might not also be reflected in the company's ESG rating. Bascially, since ESG scores lag behind the stock fluctuations due to immediate events (mergers, acqusitions, freak events like the CrowdStrike failure), any significant events that occur during 2/1/21-12/31/22 may result in stock price changes that do not perfectly correlate to ESG metrics. This could skew our correlation or regression model analysis, so we want to be careful to not falsely attributing any stock changes to ESG scores (in case of possible confounding variables). As a caveat, we should also be careful not to infer any causal relations when correlation exists.

2. ESG is a constant value that is gathered from different days for each company in 2022, though stock prices for these companies change over time every day. We cannot perform any time-series analyses with ESG due to this fact, which limits what we can do for our final phases and EDA.
   
3. Due to the nature of the Kaggle csv and yfinance data, our data is restricted to the variable types of stock data, industry type, company name, and ESG score - which actually does help us narrow down the scope of our research question, but limits the breadth of the analyses we can perform as well.
   
4. Some specific data from the yfinance library is missing - we had to drop all companies that didn't have the specified data we wanted in our specified time range. We also had to drop all companies from the original ESG csv that had missing or blank data. Overall, this means that our analyses will not be perfectly representative of all companies that use USD and have an ESG rating (can't perfectly generalize to the population). Additionally, we filtered our original ESG dataframe to be just companies traded in USD, so we can't do any inter-country comparison (although this also helps us narrow the scope of our project). Since we are also only using companies that are existent and large/mid-cap within 2/1/21 - 12/31/22, any company that stopped their operations in this time frame will be excluded. In other words, our findings might disproportionally overestimate the relationship between ESG scores and stock performance (companies that went bankrupt or have poor ESG / financial outcomes are not represented, which might skew interpretation of ESG positively).
   
5. For our sample dataset (companies_sample), the current 30 companies were chosen with a stratified sample. As a reminder, essentially, we looked at all of the different industries, and ranked them by total ESG score. then, we divided up the industries into 3 groups: high ESG score, medium ESG score, and low ESG score. Then, from each group, we chose 2 industries from a random sample. After this, we then randomly selected 5 companies from each industry, giving us a dataset of 30 companies. This sample is not fully representative of all USD-using companies with ESG ratings (this actually ties into one of our questions for reviewers). After getting feedback, we may also consider complete random sampling (no stratification) to expand the representativeness of our sample dataset, but since we are exploring potential connections between ESG ratings and company stock performance, we may need to sample not only by industry but also by ESG rating levels to ensure a more balanced and comprehensive analysis of the different ESG performance tiers (for our sample dataset). Furthermore, though our sample size is sufficient to assume normality in distribution, it is still rather small compared to merged_df's number of companies included, so our findings might reflect trends only in specific industries or companies rather than the average and general market trends.
   
6. We currently plan on comparing the rate of change of the sample stocks (in companies_sample) to the S&P 500's rate of change. We also plan on taking a look at volatility, cumulative returns, and the 50 day simple moving average, but other measures of stock performance might provide more valuable insights (but we will proceed with these 4 for now). Additionally, due to the last processing date of the ESG scores for the companies, we also restricted our stock data to be from 2/1/21 - 12/31/22, which poses a limitation on the amount of analyses we can garner as we cannot extrapolate our conclusions to beyond this time frame. 


## Preregistration Statement

### Preregistration Statement 2
#### Hypothesis 2
Industries with 'High' ESG score levels exhibit a higher correlation coefficient between their stock's rate of change over the specified time period and the ESG total score than industries with 'Low' or 'Medium' ESG score levels. We anticipate this pattern to hold across both our full dataset (`merged_df`) and our sample dataset (`sample_companies`).

#### Analysis 2
Building on findings from Friede, G., Busch, T., & Bassen, A. (2015) that suggest a positive relationship between ESG and company performance in over 2100 studies, we aim to examine whether ESG score levels correlate with stock return rate more strongly in industries with higher ESG scores. We will conduct two hypothesis tests, setting our alternative hypothesis as follows: the correlation coefficient (between stock rate of change, `percentage_change`, and total ESG score, `total_score`) is higher for industries with “High” ESG scores than for those with “Medium” or “Low” scores.

To test this, we will use a Fisher Z-transformation model to statistically compare the correlation coefficients. We’ll determine whether the difference between the 'High' ESG score group and the combined 'Medium' and 'Low' groups is significant at a 95% confidence level. One test will be applied to the full dataset (`merged_df`), and the other to the sample dataset (`sample_companies`).


---
## Data Analysis:



## Preregistration Statement 2

In [3]:
# Preregistration Statement 2

#the merged_df
# Step 1: Define the two groups based on ESG levels
pop_high = merged_df[merged_df['total_grade'] == 'BBB']
pop_not_high = merged_df[(merged_df['total_grade'] == 'BB') | (merged_df['total_grade'] == 'B')]

# Correlation between `percentage_change` and `total_score`
pop_r_high = pop_high['percentage_change'].corr(pop_high['total_score'])
pop_r_not_high = pop_not_high['percentage_change'].corr(pop_not_high['total_score'])

# Sample sizes for each group
pop_n_high = pop_high.shape[0]
pop_n_not_high = pop_not_high.shape[0]

#the sample_companies
# Step 1: Define the two groups based on ESG levels
sample_high = sample_companies[sample_companies['total_grade'] == 'BBB']
sample_not_high = sample_companies[(sample_companies['total_grade'] == 'BB') | (sample_companies['total_grade'] == 'B')]

# Correlation between `percentage_change` and `total_score`
sample_r_high = sample_high['percentage_change'].corr(sample_high['total_score'])
sample_r_not_high = sample_not_high['percentage_change'].corr(sample_not_high['total_score'])

# Sample sizes for each group
sample_n_high = sample_high.shape[0]
sample_n_not_high = sample_not_high.shape[0]



In [4]:
def z_test_correlation(n1, r1, n2, r2, alpha=0.05):
    """
    Perform a Z-test for the difference between two correlation coefficients.
    
    Parameters:
    n1 (int): Sample size of the first group
    r1 (float): Correlation coefficient of the first group
    n2 (int): Sample size of the second group
    r2 (float): Correlation coefficient of the second group
    alpha (float): Significance level for hypothesis testing (default is 0.05)
    
    Returns:
    float: Z-score for the difference in correlations
    float: p-value for the test
    str: Interpretation of the result
    """
    # Step 1: Convert correlation coefficients to Fisher's Z scores
    z1 = np.arctanh(r1)
    z2 = np.arctanh(r2)
    
    # Step 2: Calculate the standard error for the difference in Z scores
    se_diff = np.sqrt(1 / (n1 - 3) + 1 / (n2 - 3))
    
    # Step 3: Calculate the Z score for the difference between the two correlation coefficients
    z_score = (z1 - z2) / se_diff
    print(f"Z-score is : {z_score}")
    
    # Step 4: Calculate the p-value (two-tailed)
    p_value = 2 * (1 - norm.cdf(abs(z_score)))
    print(f'P value is: {p_value}')
    
    # Step 5: Interpretation
    if p_value < alpha:
        interpretation = "The difference in correlation coefficients is statistically significant."
    else:
        interpretation = "The difference in correlation coefficients is not statistically significant."
    
    return interpretation

In [5]:
#perform z-test:
z_test_correlation(pop_n_high, pop_r_high, pop_n_not_high, pop_r_not_high)


Z-score is : 1.2061190397733659
P value is: 0.22777159449607876


'The difference in correlation coefficients is not statistically significant.'

In [6]:
z_test_correlation(sample_n_high, sample_r_high, sample_n_not_high, sample_r_not_high)

Z-score is : -0.5858445861831135
P value is: 0.5579799584750469


'The difference in correlation coefficients is not statistically significant.'

## Hypo 3 (undecided)

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Step 1: Aggregate the data at the industry level
industry_data = merged_df.groupby('industry').agg({
    'total_score': 'mean',
    'percentage_change': 'mean',
    '50_day_SMA': 'mean'
}).rename(columns={'total_score': 'average_total_esg_score'})

# Step 2: Standardize the predictor (average_total_esg_score)
scaler = StandardScaler()
industry_data['average_total_esg_score_scaled'] = scaler.fit_transform(industry_data[['average_total_esg_score']])

# Step 3: Linear Regression for percent_change
X_linear = industry_data[['average_total_esg_score_scaled']]
y_linear = industry_data['percentage_change']
X_linear = sm.add_constant(X_linear)  # Add intercept

linear_model = sm.OLS(y_linear, X_linear).fit()
print("Linear Regression Results for Percent Change:")
print(linear_model.summary())

# Step 4: Logistic Regression for 50_day_SMA as binary performance (above/below median)
median_SMA = industry_data['50_day_SMA'].median()
industry_data['SMA_performance'] = (industry_data['50_day_SMA'] > median_SMA).astype(int)

X_logistic = industry_data[['average_total_esg_score_scaled']]
y_logistic = industry_data['SMA_performance']

logistic_model = LogisticRegression()
logistic_model.fit(X_logistic, y_logistic)

# Print Logistic Regression results
print("Logistic Regression Coefficients for 50-Day SMA Performance:")
print(f"Intercept: {logistic_model.intercept_[0]}")
print(f"Coefficient for ESG Score: {logistic_model.coef_[0][0]}")

# Classification report for logistic model
y_pred = logistic_model.predict(X_logistic)
print("\nClassification Report for Logistic Regression Model:")
print(classification_report(y_logistic, y_pred))


import scipy.stats as stats

# Linear Regression - Hypothesis test for βESG > 0
esg_coefficient = linear_model.params['average_total_esg_score_scaled']
esg_standard_error = linear_model.bse['average_total_esg_score_scaled']

# Calculate the t-statistic for a one-tailed test
t_statistic = esg_coefficient / esg_standard_error

# Get the p-value for the one-tailed test (βESG > 0)
p_value_one_tailed = 1 - stats.t.cdf(t_statistic, df=linear_model.df_resid)

print("Linear Regression Test for βESG > 0")
print(f"ESG Coefficient: {esg_coefficient}")
print(f"T-statistic: {t_statistic}")
print(f"One-tailed p-value: {p_value_one_tailed}")

if p_value_one_tailed < 0.05:
    print("Reject the null hypothesis: ESG has a positive effect on percent change.")
else:
    print("Fail to reject the null hypothesis: No significant positive effect of ESG on percent change.")


# Logistic Regression using statsmodels for p-values
X_logistic_sm = sm.add_constant(X_logistic)  # Add intercept for statsmodels
logit_model = sm.Logit(y_logistic, X_logistic_sm).fit()
print(logit_model.summary())

# Extract one-tailed p-value for ESG coefficient
esg_logit_p_value_one_tailed = logit_model.pvalues['average_total_esg_score_scaled'] / 2
print(f"One-tailed p-value for ESG coefficient in Logistic Regression: {esg_logit_p_value_one_tailed}")

if esg_logit_p_value_one_tailed < 0.05:
    print("Reject the null hypothesis: ESG has a positive effect on 50-Day SMA performance classification.")
else:
    print("Fail to reject the null hypothesis: No significant positive effect of ESG on 50-Day SMA performance classification.")



Linear Regression Results for Percent Change:
                            OLS Regression Results                            
Dep. Variable:      percentage_change   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     13.07
Date:                Thu, 14 Nov 2024   Prob (F-statistic):           0.000812
Time:                        16:12:09   Log-Likelihood:                -199.69
No. Observations:                  43   AIC:                             403.4
Df Residuals:                      41   BIC:                             406.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [7]:
#the merged_df
#mean
pop_return_high = pop_high['percentage_change'].mean()
pop_return_not_high = pop_not_high['percentage_change'].mean()
#std

pop_return_high_std = pop_high['percentage_change'].std()
pop_return_not_high_std = pop_not_high['percentage_change'].std()

# Sample sizes for each group
pop_n_high = pop_high.shape[0]
pop_n_not_high = pop_not_high.shape[0]

In [8]:
def z_test_mean_difference(mean1, std1, n1, mean2, std2, n2, alpha=0.05):
    """
    Perform a Z-test for the difference between two means.
    
    Parameters:
    mean1 (float): Mean of the first group
    std1 (float): Standard deviation of the first group
    n1 (int): Sample size of the first group
    mean2 (float): Mean of the second group
    std2 (float): Standard deviation of the second group
    n2 (int): Sample size of the second group
    alpha (float): Significance level for hypothesis testing (default is 0.05)
    
    Returns:
    float: Z-score for the difference in means
    float: p-value for the test
    str: Interpretation of the result
    """
    # Step 1: Calculate the standard error of the difference between the means
    se_diff = np.sqrt((std1**2 / n1) + (std2**2 / n2))
    
    # Step 2: Calculate the Z score for the difference between the two means
    z_score = (mean1 - mean2) / se_diff
    
    # Step 3: Calculate the p-value (two-tailed test)
    p_value = 2 * (1 - norm.cdf(abs(z_score)))
    
    # Step 4: Interpretation
    if p_value < alpha:
        interpretation = "The difference in means is statistically significant."
    else:
        interpretation = "The difference in means is not statistically significant."
    
    return z_score, p_value, interpretation

In [9]:
#Hypothesis 3 test 
z_test_mean_difference(pop_return_high, pop_return_high_std, pop_n_high, pop_return_not_high, pop_return_not_high_std, pop_n_not_high)


(3.8393115812888556,
 0.00012337977253773502,
 'The difference in means is statistically significant.')

---
## Evaluation of significance

### Hypothesis 2:
The Z-test results shows that, the p-values for both the full population (merged_df) and sample datasets (sample_companies) are not statistically significant at a 5% significance level (p-values are 0.2278 for population test and 0.5580 for sample test). This suggests that we fail to reject the null hypothesis and conclude that there is no significant difference in the correlation coefficients between the "High" ESG group and the combined "Medium" and "Low" ESG groups. 



--- 
## Conclusions

---
## Limitations 

---
## Acknowledgements and Bibliography 

### part one - exploring average environmental, social, governance, and total ESG scores by industry

##### total esg score by industry

In [ ]:
plt.figure(figsize = (14, 10))
sns.barplot(x = 'Average Total ESG Score', y = 'Industry', data = avg_esg_by_industry, color = "#b97df3")
plt.title('Average Total ESG Score by Industry', horizontalalignment = 'center', fontsize = 16, fontweight = 'bold', )
plt.xlabel('Average Total ESG Score', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')
plt.ylabel('Industry Name', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')
plt.show()

interestingly (and somewhat predictably) - the industries with the lowest ESG scores are Leisure Products, Airlines, Marine, Biotechnology, and Banking. the industries with the highest ESG scores are Utilities, Tobacco, Industrial Conglomerates, Packaging, and Industrial Conglomerates. This is relevant to our research question as we wanted to explore associations between industry and ESG ranking, so it will also be interesting to see if there are differences in this ranking when dividing ESG into its component parts.

##### environmental score by industry

In [ ]:
# enviro ranking
avg_enviro_by_industry = merged_df.groupby('industry')['environment_score'].mean().reset_index()
avg_enviro_by_industry.columns = ['Industry', 'Average Environmental Score']

avg_enviro_by_industry = avg_enviro_by_industry.sort_values(by = 'Average Environmental Score', ascending = False)
print("best average Environment scores")
print(avg_enviro_by_industry.head(6))

avg_enviro_by_industry = avg_enviro_by_industry.sort_values(by = 'Average Environmental Score', ascending = True)
print("\nworst average Environment scores")
print(avg_enviro_by_industry.head(6))

##### social score by industry

In [ ]:
# social ranking
avg_social_by_industry = merged_df.groupby('industry')['social_score'].mean().reset_index()
avg_social_by_industry.columns = ['Industry', 'Average Social Score']

avg_social_by_industry = avg_social_by_industry.sort_values(by = 'Average Social Score', ascending = False)
print("best average social scores")
print(avg_social_by_industry.head(6))

avg_social_by_industry = avg_social_by_industry.sort_values(by = 'Average Social Score', ascending = True)
print("\nworst average social scores")
print(avg_social_by_industry.head(6))

##### governance score by industry

In [ ]:
# gov ranking
avg_gov_by_industry = merged_df.groupby('industry')['governance_score'].mean().reset_index()
avg_gov_by_industry.columns = ['Industry', 'Average Governance Score']

avg_gov_by_industry = avg_gov_by_industry.sort_values(by = 'Average Governance Score', ascending = False)
print("best average governance scores")
print(avg_gov_by_industry.head(6))

avg_gov_by_industry = avg_gov_by_industry.sort_values(by = 'Average Governance Score', ascending = True)
print("\nworst average governance scores")
print(avg_gov_by_industry.head(6))

##### environmental, social, and governance scores by industry

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (24, 10)) 

# Plot 1: Average Environmental Score by Industry
sns.barplot(x = 'Average Environmental Score', y = 'Industry', data = avg_enviro_by_industry, color = "#39c988", ax = axes[0])
axes[0].set_title('Average Environmental Score by Industry', horizontalalignment = 'center', fontsize = 16, fontweight = 'bold')
axes[0].set_xlabel('Average Environmental Score', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')
axes[0].set_ylabel('Industry Name', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')

# Plot 2: Average Social Score by Industry
sns.barplot(x ='Average Social Score', y = 'Industry', data = avg_social_by_industry, color = "#13bcd8", ax = axes[1])
axes[1].set_title('Average Social Score by Industry', horizontalalignment = 'center', fontsize = 16, fontweight = 'bold')
axes[1].set_xlabel('Average Social Score', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')
axes[1].set_ylabel('') 

# Plot 3: Average Governance Score by Industry
sns.barplot(x ='Average Governance Score', y = 'Industry', data = avg_gov_by_industry, color = "#df3e5d", ax = axes[2])
axes[2].set_title('Average Governance Score by Industry', horizontalalignment = 'center', fontsize = 16, fontweight = 'bold')
axes[2].set_xlabel('Average Governance Score', fontsize = 14, fontweight = 'bold', horizontalalignment = 'center')
axes[2].set_ylabel('') 

plt.tight_layout()
plt.show()

In [ ]:
avg_enviro_by_industry_ = avg_enviro_by_industry.rename(columns = {'Average Environmental Score': 'Environmental'})
avg_social_by_industry_ = avg_social_by_industry.rename(columns = {'Average Social Score': 'Social'})
avg_gov_by_industry_ = avg_gov_by_industry.rename(columns = {'Average Governance Score': 'Governance'})
stacked_data = avg_enviro_by_industry_.merge(avg_social_by_industry_, on='Industry').merge(avg_gov_by_industry_, on='Industry')

fig, ax = plt.subplots()
stacked_data.set_index('Industry').plot(kind = 'bar', stacked = True, ax = ax, color = ['#39c988', '#13bcd8', '#df3e5d'])
ax.set_title('Average ESG Scores by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
ax.set_xlabel('Industry Name', fontsize = 14, fontweight = 'bold')
ax.set_ylabel('Score', fontsize = 14, fontweight = 'bold')
plt.xticks(rotation = 90)  
plt.legend(title = 'Score Type')
plt.tight_layout()
plt.show()

the industries with the lowest Environmental scores are also Leisure Products, Marine, Biotechnology, Trading Companies & Distributors, and Airlines. the industries with the highest Environmental scores are also Utilities, Industrial Conglomerates, Packaging, Distributors, and Energy. 

the industries with the lowest Social scores are Automobiles, Leisure Products, Airlines, Biotechnology, and Communications. the industries with the highest Social scores are also Utilities, Trading Companies & Distributors, Road & Rail, Tobacco, and Packaging. 

the industries with the lowest Governance scores are Airlines, Trading Companies & Distributors, Automobiles, Diversified Consumer Services, and Hotels, Restaurants, & Leisure. the industries with the highest Governance scores are also Tobacco, Distributors, Industrial Conglomerates, Food Products, and Tobacco.

overall, it looks like Airlines is an industry that performs especially bad in all 3 metrics, and Distributors does pretty well in all 3 metrics. 

---
### part two - distribution of ESG scores, with respect to grade and level


#### initial: esg scores across all companies

In [ ]:
plt.figure(figsize = (10, 8))
sns.histplot(merged_df['environment_score'], kde = True, color = '#39c988', label = 'Environmental Score')
sns.histplot(merged_df['social_score'], kde = True, color = '#13bcd8', label = 'Social Score')
sns.histplot(merged_df['governance_score'], kde = True, color = '#df3e5d', label = 'Governance Score')
#sns.histplot(merged_df['total_score'], kde = True, color = '#b97df3', label = 'Total ESG Score')

plt.title('ESG Metrics Across All Companies', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xlabel('Score', fontsize = 14, fontweight = 'bold')
plt.ylabel('Frequency', fontsize = 14, fontweight = 'bold')
plt.legend()
plt.show()

This plot highlights the distribution of Environmental, Social, Governance (E/S/G) scores and the total ESG score across companies. A clear bimodal pattern in the total ESG score (purple) suggests two main groups of companies — one with lower scores around 600-800 and another with higher scores around 1000-1200. In terms of the indivisual ESG socres, the Environmental score (green) shows the widest spread, indicating greater variability than Social and Governance scores (blue and red), which are more concentrated. This plot provides information about ESG scores distribution, giving us some general understanding about ESG score in our datasets.      

#### distribution of ESG wrt Level and Industry

In [ ]:
# Count plot for Environmental Level
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'environment_level', palette = 'Greens_d', hue_order = ['Medium', 'High', 'Excellent'])
plt.title('Environmental Level by Industry', horizontalalignment = 'center', fontweight = 'bold', fontsize = 16)
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', horizontalalignment = 'center', fontweight = 'bold', fontsize = 14)
plt.ylabel('Count', horizontalalignment = 'center', fontweight = 'bold', fontsize = 14)
plt.legend(title = 'Environmental Level')
plt.show()

The plot shows distribution of Environmental levels for each industry. In terms of coloration, the darker the color is, the higher Environmental level is indicated. As indicated by the plot, the distribution of companies in each Environmental level are relatively wide spreaded, in which most of the companies has "Medium" or "High" Environmental levels, but the population with "High" environmental level are not extremely low. Overall, this plot provides an overview about the Environmental level across industries.     

In [ ]:
# Count plot for Social Level
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'social_level', palette = 'Blues_d', hue_order = ['Low','Medium', 'High', 'Excellent'])
plt.title('Social Level by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', horizontalalignment = 'center', fontweight = 'bold', fontsize = 14)
plt.ylabel('Count', horizontalalignment = 'center', fontweight = 'bold', fontsize = 14)
plt.legend(title = 'Social Level')
plt.show()

The plot shows distribution of Social levels for each industry. In terms of coloration, the darker the color is, the higher Social level is indicated. Based on the graph, most of the companies in the population has "Medium" or "High" social levels and just a few companies (less then 10) has "Excellent" or "Low" Social levels. 

In [ ]:
# Count plot for Governance Level
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'governance_level', palette = 'Reds_d', hue_order = ['Low','Medium', 'High'])
plt.title('Governance Level by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', fontsize = 14, fontweight = 'bold')
plt.ylabel('Count', fontsize = 14, fontweight = 'bold')
plt.legend(title = 'Governance Level')
plt.show()

The plot shows distribution of Governance levels for each industry. In terms of coloration, the darker the color is, the higher Governance level is indicated. Basd on the graph, most of the companies in the population has "Medium" Governance levels and just 5 has "High" and only 1 "Low" Governance levels, which indicates more concentrated Governance Level comparing to Social or Environmental Levels.     

#### distribution of ESG wrt Grade and Industry

In [ ]:
# Bar plot for Environmental Grade
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'environment_grade', palette = 'Greens_d', hue_order = ['B','BB', 'BBB', 'A','AA'])
plt.title('Environmental Grade by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', fontsize = 14, fontweight = 'bold')
plt.ylabel('Count', fontsize = 14, fontweight = 'bold')
plt.legend(title = 'Environmental Grade')
plt.show()

In [ ]:
# Bar plot for Social Grade
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'social_grade', palette = 'Blues_d', hue_order = ['CCC','B','BB', 'BBB', 'A','AA'])
plt.title('Social Grade by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', fontsize = 14, fontweight = 'bold')
plt.ylabel('Count', fontsize = 14, fontweight = 'bold')
plt.legend(title = 'Social Grade')
plt.show()

In [ ]:
# Bar plot for Governance Grade
plt.figure(figsize = (12, 6))
sns.countplot(data = merged_df, x = 'industry', hue = 'governance_grade', palette = 'Reds_d', hue_order = ['C','B','BB', 'BBB'])
plt.title('Governance Grade by Industry', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(rotation = 90)
plt.xlabel('Industry Name', fontsize = 14, fontweight = 'bold')
plt.ylabel('Count', fontsize = 14, fontweight = 'bold')
plt.legend(title='Governance Grade')
plt.show()

The bar plots showing ESG grades follow a similar pattern to those with ESG levels but provide more detailed ratings. We are keeping these plots for potential use in future research, in case the ESG grades become relevant for further analysis. 

---
### part three - linear regression amongst ESG and stock percentage change

In [ ]:
X = merged_df[['environment_score', 'social_score','governance_score']]   
y = merged_df['percentage_change']  

model = LinearRegression().fit(X, y)
print(f"environmental score coefficient: {model.coef_[0]}")
print(f"social score coefficient: {model.coef_[1]}")
print(f"governance score coefficient: {model.coef_[2]}")
print(f"intercept: {model.intercept_}")

For every 1-unit increase in the Environmental score (assuming all other variables remain constant), the stock percentage change is expected to increase by 0.1079 units. The positive coefficient suggests that higher Environmental scores are associated with better stock performance (or higher returns).

For every 1-unit increase in the Social score (assuming all other variables remain constant), the stock percentage change is expected to decrease by 0.0237 units. The negative coefficient indicates that better Social scores might be associated with lower stock performance, but generally, since the coefficient is so close to 0, Social scores seem to have little impact on stock performance.

For every 1-unit increase in the Governance score (assuming all other variables remain constant), the stock percentage change is expected to decrease by 0.0750 units. This negative coefficient suggests that improvements in governance (e.g., stricter regulation or more ethical practices) are associated with slightly lower stock returns. This could imply that governance improvements come at a financial cost.

---
### part four - distribution of ESG and financial metrics

#### ESG vs financial metric wrt Industry

In [ ]:
fig, axes = plt.subplots(3, 1, figsize = (10, 10))
palette = sns.color_palette('tab20', n_colors = merged_df['industry'].nunique())
distinct_colors = sns.color_palette("hsv", n_colors=43)

# Scatter plot: Percentage Change vs. Total ESG Score (with legend)
sns.scatterplot(data = merged_df, x = 'total_score', y = 'percentage_change', hue = 'industry', palette = distinct_colors, ax = axes[0])
axes[0].set_title("Percentage Change vs Total ESG Score", fontsize = 14, horizontalalignment = 'center', fontweight = 'bold')
axes[0].set_xlabel("Total ESG Score", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')
axes[0].set_ylabel("Percentage Change in Stock Price", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')
axes[0].legend(title='Industry',bbox_to_anchor = (1.05, 1), loc = 'upper left', fontsize = 'small')

# Scatter plot: Volatility vs. Total ESG Score (no legend)
sns.scatterplot(data = merged_df, x = 'total_score', y = 'volatility', hue = 'industry', palette = distinct_colors, ax = axes[1], legend = False)
axes[1].set_xlabel("Total ESG Score", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')
axes[1].set_ylabel("Stock Volatility", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')

# Scatter plot: Cumulative Return vs. Total ESG Score (no legend)
sns.scatterplot(data = merged_df, x = 'total_score', y = 'cumulative_return', hue = 'industry', palette = distinct_colors, ax = axes[2], legend = False)
axes[2].set_title("Cumulative Return vs Total ESG Score", fontsize = 14, horizontalalignment = 'center', fontweight = 'bold')
axes[2].set_xlabel("Total ESG Score", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')
axes[2].set_ylabel("Cumulative Return", fontsize = 11, fontweight = 'bold', horizontalalignment = 'center')

plt.subplots_adjust(hspace = 0.5)
plt.show()
plt.clf()


1. Percentage Change vs Total ESG Score: The first plot displays the relationship between Total ESG Score and percentage change in stock price. The data points are widely scattered, indicating no clear correlation between ESG scores and drastic stock price changes across industries. However, companies with higher ESG scores tend to have less extreme changes in stock prices, suggesting stability.   
   
2. Volatility vs Total ESG Score: The second plot displays the stock volatility as a function of Total ESG Score. Overall, companies with higher ESG scores appear to have lower volatility, reflecting that higher ESG performance might be associated with less risky, more stable stock behavior.   
   
3. Cumulative Return vs Total ESG Score: The third plot displays  the cumulative return relative to Total ESG Score. This plot is very similar to the first plot with Percentage Change vs Total ESG Score, suggesting Cumulative Return can be removed as a potential influential variable.   

#### total ESG score vs financial metric with unnamed industry

In [ ]:
industry_avg = merged_df.groupby('industry').agg({
    'environment_score': 'mean',
    'social_score': 'mean',
    'governance_score': 'mean',
    'total_score': lambda x: round(x.mean()),  
    'percentage_change': 'mean',
    'volatility': 'mean',
    'cumulative_return': 'mean'}).reset_index()

fig, axes = plt.subplots(1, 3, figsize = (18, 6)) 

# Average Percentage Change by Industry vs. Total ESG Score
sns.barplot(data = industry_avg, x = 'total_score', y = 'percentage_change', hue = 'total_score', palette = 'Blues_d', ax = axes[0], legend = False)
axes[0].set_title("Average Percentage Change by Industry vs. Total ESG Score", fontweight = 'bold', horizontalalignment = 'center', size = 15)
axes[0].set_xlabel("Average Total ESG Score",  fontweight = 'bold', horizontalalignment = 'center')
axes[0].set_ylabel("Average Percentage Change in Stock Price",  fontweight = 'bold', horizontalalignment = 'center')
axes[0].tick_params(axis = 'x', rotation = 90)

# Average Volatility by Industry vs. Total ESG Score
sns.barplot(data = industry_avg, x = 'total_score', y = 'volatility', hue = 'total_score', palette = 'Reds_d', ax = axes[1], legend = False)
axes[1].set_title("Average Volatility by Industry vs. Total ESG Score", fontweight = 'bold', horizontalalignment = 'center', size = 15)
axes[1].set_xlabel("Average Total ESG Score", fontweight = 'bold', horizontalalignment = 'center')
axes[1].set_ylabel("Average Volatility",  fontweight = 'bold', horizontalalignment = 'center')
axes[1].tick_params(axis = 'x', rotation = 90)

# Average Cumulative Return by Industry vs. Total ESG Score
sns.barplot(data = industry_avg, x = 'total_score', y = 'cumulative_return', hue = 'total_score', palette = 'Greens_d', ax = axes[2], legend = False)
axes[2].set_title("Average Cumulative Return by Industry vs. Total ESG Score", fontweight = 'bold', horizontalalignment = 'center', size = 15)
axes[2].set_xlabel("Average Total ESG Score", fontweight = 'bold', horizontalalignment = 'center')
axes[2].set_ylabel("Average Cumulative Return", fontweight = 'bold', horizontalalignment = 'center')
axes[2].tick_params(axis = 'x', rotation = 90)

plt.tight_layout()
plt.show()

1. Average Percentage Change by Industry vs. Total ESG Score: This plot shows the average percentage change in stock price for each industry relative to its average Total ESG Score. Based on the plot, industries with lower ESG scores tend to exhibit more negative price changes, while those with higher ESG scores generally show more positive price changes.   
   
2. Average Volatility by Industry vs. Total ESG Score: The second plot shows the average stock volatility for each industry compared to its Total ESG Score. The plot illustrates a decreasing pattern, in which when industries have lower ESG scores usually exhibit greater volatility, suggesting that companies with lower ESG performance may experience more unpredictable stock behavior. In contrast, higher ESG scores are associated with relatively lower volatility.   
   
3. Average Cumulative Return by Industry vs. Total ESG Score: This plot shows the relationship between cumulative stock returns and Total ESG Score by industry. Trends illustrated in this plot is similar to the first plot, again, indicating Cumulative Return might not be a significant variable if we already included Percentage Change in our model.   

--- 
### part five - summary statistics and correlation analysis

In [ ]:
# Summary statistics
summary_stats = merged_df[['environment_score', 'social_score', 'governance_score', 'total_score', 
                           'percentage_change', 'volatility', 'cumulative_return']].describe()
print(summary_stats)

# Correlation matrix
correlation_matrix = merged_df[['environment_score', 'social_score', 'governance_score', 'total_score', 
                                'percentage_change', 'volatility', 'cumulative_return']].corr()

plt.figure(figsize = (10, 6))
sns.heatmap(correlation_matrix, annot = True, cmap = 'coolwarm', linewidths = 0.5, annot_kws = {"size": 12, "fontweight": 'bold'})
plt.title('Correlation Matrix between ESG and Financial Metrics', fontsize = 16, fontweight = 'bold', horizontalalignment = 'center')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.show()

The color intensity represents the strength of the correlation, with red indicating positive correlation and blue indicating negative correlation.    
    
Based on the Correlation matrix, strong positive correlation is illustrated between the E/S/G and total ESG scores, indictaed by intermediate red color. However, the correlation appear to be relatively weak between ESG scores and stock performance measurements:    

Percentage Change: ESG scores have weak positive correlations with stock price percentage changes, with environmental and total scores showing the strongest relationships (0.23 and 0.20, respectively).   
Volatility: All ESG scores are negatively correlated with volatility. The environmental score (-0.34) and total ESG score (-0.30) show the strongest negative correlations, suggesting that higher ESG performance is associated with lower stock price volatility.   
Cumulative Return: ESG scores have weak positive correlations with cumulative return, with environmental and total scores showing the most significant relationships (0.23 and 0.20), implying that higher ESG scores may be linked to better long-term returns.   


--- 
### part six - linear and logistic regression with predictions

In [ ]:
# Linear Regression - Predicting Stock Performance (percentage_change) based on ESG Scores
X = merged_df[['environment_score', 'social_score', 'governance_score', 'total_score']]
y = merged_df['percentage_change']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression: \nRoot Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

plt.figure(figsize = (8, 6))
plt.scatter(y_test, y_pred, color = 'blue', alpha = 0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color = 'purple', linestyle = '--')
plt.title('Actual vs Predicted Stock Performance wrt Linear Regression', size = 16, fontweight = 'bold', ha = 'center')
plt.xlabel('Actual Stock Percentage Change', size = 14, fontweight = 'bold', ha = 'center')
plt.ylabel('Predicted Stock Percentage Change', size = 14, fontweight = 'bold', ha = 'center')
plt.show()

# Logistic Regression - Binary Stock Increase/Decrease
merged_df['stock_increase'] = np.where(merged_df['percentage_change'] > 0, 1, 0)
X = merged_df[['environment_score', 'social_score', 'governance_score', 'total_score']]
y = merged_df['stock_increase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy: {accuracy}")
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const)
results = model.fit()
print(results.summary())

We are building two models to predict stock performance based on ESG (Environmental, Social, and Governance) scores:   
   
Linear Regression: With a low R-squared value of 0.06 and a relatively high RMSE of 50.03, the model shows that ESG scores have limited predictive power for percentage change in stock prices. The model struggles to explain the majority of stock price variability.   
    
Logistic Regression: The model's accuracy of 58% and precision/recall metrics show that ESG scores alone do not effectively predict whether a stock will increase or decrease in value.   

The perfornace of both Linear Regression and Logistic Regression indicates we might need to explore other regression methods in interepreting companies' ESG measurements and their stock performances. 

--- 
### part seven - time series analysis of sample company stock percentage changes

In [ ]:
data_with_sp500 = yf.download(list(sample_companies['ticker']) + ['^GSPC'], start = '2021-02-01', end = '2022-12-31', progress = False)['Adj Close']
data_with_sp500.rename(columns = {'^GSPC': 'SP500'}, inplace = True)
data_without_sp500 = yf.download(list(sample_companies['ticker']), start = '2021-02-01', end = '2022-12-31', progress = False)['Adj Close']
fig, axes = plt.subplots(1, 2, figsize = (20, 8))  

# Plot WITH S&P 500
colors_with_sp500 = sns.color_palette('husl', len(data_with_sp500.columns))
for i, ticker in enumerate(data_with_sp500.columns):
    axes[0].plot(data_with_sp500.index, data_with_sp500[ticker], label = ticker, color = colors_with_sp500[i])
axes[0].set_xlabel('Date', fontweight = 'bold', ha = 'center', size = 14)
axes[0].set_ylabel('Adjusted Closing Price (in USD)', fontweight = 'bold', ha = 'center', size = 14)
axes[0].set_title('Stock Price Fluctuations for Sample Companies (Including SP500)', fontweight = 'bold', ha = 'center', size = 16)
axes[0].legend(loc = 'upper left', fontsize = 'small')
axes[0].tick_params(axis = 'x', rotation = 45)

# Plot WITHOUT S&P 500
colors_without_sp500 = sns.color_palette('husl', len(data_without_sp500.columns))
for i, ticker in enumerate(data_without_sp500.columns):
    axes[1].plot(data_without_sp500.index, data_without_sp500[ticker], label = ticker, color = colors_without_sp500[i])
axes[1].set_xlabel('Date', fontweight = 'bold', ha = 'center', size = 14)
axes[1].set_ylabel('Adjusted Closing Price (in USD)', fontweight = 'bold', ha = 'center', size = 14)
axes[1].set_title('Stock Price Fluctuations for Sample Companies (Without SP500)', fontweight = 'bold', ha = 'center', size = 16)
axes[1].legend(loc = 'upper left', fontsize = 'small')
axes[1].tick_params(axis = 'x', rotation = 45)

plt.tight_layout()
plt.show()

These side-by-side line plots illustrate the stock price changes over time for selected companies, both with and without the inclusion of the S&P 500 index (^GSPC), from February 2021 to December 2022.   
    
Left Plot (Including S&P 500): This plot shows the adjusted closing prices for the selected companies alongside the S&P 500 index. The S&P 500 (pink line) dominates the chart due to its much higher adjusted closing price compared to individual companies. This makes the comparison between the companies harder to interpret visually because the large scale of the S&P 500 price compresses the other stock prices.   
    
Right Plot (Without S&P 500): This plot excludes the S&P 500, making the individual stock prices of the selected companies easier to differentiate. With the S&P 500 removed, we can observe the individual trends more clearly, including which companies experienced significant growth or fluctuations during the observed period. Notable trends include some companies showing relatively stable prices, while others display more pronounced volatility.


---
### __Questions for Reviewers:__

1. Does it seem like we have enough columns in merged_df and sample_companies to satisfy the complexity requirement for the project? Does our research question also seem complex enough given our EDA?
   
2. Is there any advice or recommended steps to follow in creating our sample_companies dataset from the population? If not, is our method of sampling acceptable? We wanted to get some feedback before proceeding with a lot of analyses for sample_companies, so the bulk of our EDA involves merged_df (as a refresher - we have one big dataset, merged_df with 600 companies that we did some EDA with, but we also want to include a smaller sample dataset so we can look at some individual companies as well. the purpose of this smaller dataset would be then to observe trends on a company-level, and study the metrics of our research question on this smaller scale). Is our sample size of 30 also acceptable?
   
3. How many visualizations (or like data analysis chunks) are recommended for the final project? (ballpark range would be helpful, or if there's any better way of quantifying, or will we get to look at some examples of a completed project later?)

4. Do the visualizations we currently have seem like they're on the right path for the final phases? Is there anything glaringly obvious that should be changed or labeled better?
 
5. Regarding the visualizations and stuff we have made for our EDA so far: should we further explore these specific visualizations more in depth? Or should we expand our DA to other variables in the datasets that we maybe haven't used yet?

6. We were looking at the guidelines for phase III as well - are you able to give us an example of a 'sample analysis' that we should aim to present in that part, based on the EDA that we currently have done? We are little confused about what that entails. 